In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [9]:
(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


In [10]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


In [11]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)


In [12]:
'''
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
'''
base_model = keras.applications.ResNet50(
    weights="imagenet",
    input_shape=(32,32,3),
    include_top=False
)

In [13]:
base_model.trainable = False


inputs = keras.Input(shape=(32, 32, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(10)(x)
model = keras.Model(inputs, outputs)


In [14]:
#Traditional Training
#model = tf.keras.models.Sequential([
#  tf.keras.layers.Flatten(input_shape=(32, 32, 3)),
#  tf.keras.layers.Dense(128, activation='relu'),
#  tf.keras.layers.Dense(10)
#])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')],
)

model.fit(
    ds_train,
    epochs=20,
    validation_data=ds_test,
)


Epoch 1/20
391/391 [==============================] - 27s 62ms/step - loss: 2.1519 - accuracy: 0.2144 - val_loss: 2.0233 - val_accuracy: 0.2694
Epoch 2/20
391/391 [==============================] - 25s 63ms/step - loss: 1.9827 - accuracy: 0.2892 - val_loss: 1.9270 - val_accuracy: 0.3102
Epoch 3/20
391/391 [==============================] - 26s 67ms/step - loss: 1.9214 - accuracy: 0.3158 - val_loss: 1.9295 - val_accuracy: 0.3078
Epoch 4/20
391/391 [==============================] - 26s 66ms/step - loss: 1.8851 - accuracy: 0.3278 - val_loss: 1.8558 - val_accuracy: 0.3377
Epoch 5/20
391/391 [==============================] - 24s 62ms/step - loss: 1.8563 - accuracy: 0.3368 - val_loss: 1.8236 - val_accuracy: 0.3556
Epoch 6/20
391/391 [==============================] - 25s 64ms/step - loss: 1.8352 - accuracy: 0.3485 - val_loss: 1.8173 - val_accuracy: 0.3618
Epoch 7/20
391/391 [==============================] - 25s 63ms/step - loss: 1.8144 - accuracy: 0.3571 - val_loss: 1.7948 - val_accuracy:

In [16]:
# Unfreeze the base model
base_model.trainable = True

# It's important to recompile your model after you make any changes
# to the `trainable` attribute of any inner layer, so that your changes
# are take into account
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')],
)
testing = model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
)



Epoch 1/10
391/391 [==============================] - 855s 2s/step - loss: 3.9437 - accuracy: 0.1007 - val_loss: 2.3037 - val_accuracy: 0.1000
Epoch 2/10
339/391 [=========================>....] - ETA: 1:52 - loss: 2.3033 - accuracy: 0.0979

KeyboardInterrupt: 

In [ ]:

plt.plot(
    np.arange(1, 11), 
    testing.history['accuracy'], label='Accuracy'
)
plt.show

In [ ]:
plt.plot(
    np.arange(1, 11), 
    testing.history['loss'], label='Loss'
)
plt.show

In [ ]:
model.evaluate(ds_test)
